In [1]:
!pip install numpy
!pip install nltk
!pip install emoji
!pip install pandas

In [2]:
import emoji
import os
import glob
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
path = r"E:\twitter_SA\tweets"

# Check current working directory.
retval = os.getcwd()
print("Current working directory %s" % retval)

# Now change the directory
os.chdir(path)

# Check current working directory.
retval = os.getcwd()

print("Directory changed successfully %s" % retval)

Current working directory C:\Users\user\Untitled Folder 3
Directory changed successfully E:\twitter_SA\tweets


In [4]:
#Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [5]:

#combine all files in the list
df_tweets = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [6]:
#Check first 5 rows of df_tweets dataset
df_tweets.head()

,SN,Text,Username,Location,Date,Topic
0,0,I am having a hard time with #Lent this year ...,wagagaikoko,NaN,3/10/2021 19:10,corona
1,1,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",RZgheib7,NaN,3/10/2021 19:10,corona
2,2,I had the Corona already last January. So who...,cestlaviv,Carinthia - Austria,3/10/2021 19:10,corona
3,3,"Ooohh karma, you wonderful thing,man's restric...",FlavionsKimani,Nairobi Kenya,3/10/2021 19:10,corona
4,4,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",Zizo90693787,NaN,3/10/2021 19:10,corona


In [7]:
df_tweets.shape

(23000, 6)

In [8]:
#Get summary of df_tweets dataframe
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SN        23000 non-null  int64 
 1   Text      23000 non-null  object
 2   Username  23000 non-null  object
 3   Location  15126 non-null  object
 4   Date      23000 non-null  object
 5   Topic     23000 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [9]:
#Check sum of missing values in each column of df_tweets dataframe
df_tweets.isnull().sum()

SN             0
Text           0
Username       0
Location    7874
Date           0
Topic          0
dtype: int64

In [10]:
# Convert created_at and account_created_at variables from object to datetime64 
df_tweets['Date'] = df_tweets['Date'].astype('datetime64')
# Check for earliest and latest tweet
print("Earliest tweet: ", df_tweets['Date'].min())
print("Latest tweet: ", df_tweets['Date'].max(), '\n')

Earliest tweet:  2021-03-03 14:10:00
Latest tweet:  2021-03-11 17:24:00 



In [11]:
# function for converting emoticons to text
emoticons = {
':-)': 'happy / smile', 
':)': 'happy / smile', 
';)': 'wink / glad', 
':o)': 'happy / smile', 
':]': 'happy / smile', 
':3': 'happy / smile', 
':c)': 'happy / smile',
':>': 'happy / smile', 
'=]': 'happy / smile', 
'8)': 'happy / smile', 
'=)': 'happy / smile', 
':}': 'happy / smile',
':^)': 'happy / smile', 
':-D': 'laugh / big grin',
':D': 'laugh / big grin',
'8-D': 'laugh / big grin / laugh with glasses / wide-eyed surprise', 
'8D': 'laugh / big grin / laugh with glasses / wide-eyed surprise', 
'x-D': 'laugh', 
'xD': 'laugh', 
'X-D': 'laugh', 
'XD': 'laugh', 
'=-D': 'laugh / big grin', 
'=D': 'laugh / big grin',
'=-3': 'laugh / big grin', 
'=3': 'laugh / big grin', 
':-))': 'very happy / double chin', 
":'-)": 'tears of happiness', 
":')": 'tears of happiness', 
':*': 'kiss', 
':^*': 'kiss', 
'>:P': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':-P': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':P': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
'X-P': 'tongue sticking out / cheeky / playful / blowing a raspberry',
'x-p': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
'xp': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
'XP': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':-p': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':p': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
'=p': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':-b': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
':b': 'tongue sticking out / cheeky / playful / blowing a raspberry', 
'>:)': 'devilish / cheeky / playful', 
'>;)': 'devilish / cheeky / playful / wink', 
'>:-)': 'devilish / cheeky / playful',
'<3': 'heart / love',
':L': 'skeptical / undecided / uneasy / hesitant', 
':-/': 'skeptical / undecided / uneasy / hesitant', 
'>:/': 'skeptical / annoyed / undecided / uneasy / hesitant', 
':S': 'skeptical / undecided / uneasy / hesitant', 
'>:[': 'frown / angry / pouting', 
':@': 'frown / sad / pouting', 
':-(': 'frown / sad / pouting', 
':[': 'frown / sad / pouting', 
':-||': 'frown / pouting', 
'=L': 'skeptical / undecided / uneasy / hesitant', 
':<': 'frown / sad / pouting',
':-[': 'frown / sad / pouting', 
':-<': 'frown / sad / pouting', 
'=\\': 'skeptical / undecided / uneasy / hesitant', 
'=/': 'skeptical / undecided / uneasy / hesitant', 
'>:(': 'skeptical / annoyed / undecided / uneasy / hesitant', 
':(': 'frown / sad / pouting', 
'>.<': 'frown / pouting', 
":'-(": 'cry', 
":'(": 'cry', 
':\\': 'skeptical / undecided / uneasy / hesitant', 
':-c': 'frown / sad / pouting',
':c': 'frown / sad / pouting', 
':{': 'frown / sad / pouting', 
'>:\\': 'skeptical / annoyed / undecided / uneasy / hesitant', 
';(': 'skeptical / annoyed / undecided / uneasy / hesitant'
}

In [12]:
def emoticon_text(text):
    words=text.split()
    reformed = [emoticons[word] if word in emoticons else word for word in words]
    text = " ".join(reformed)
    return text

In [13]:
#Apply converting emoticons to text function to text column in df_tweets dataframe
df_tweets['cleaned_text'] = df_tweets['Text'].apply(lambda x: emoticon_text(x))

In [14]:
# function for replacing contractions
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [19]:
def replace_contractions(text):
    text = text.replace("’","'")
    words = text.split()
    reformed = [contractions[word] if word in contractions else word for word in words]
    text = " ".join(reformed)
    return text

In [20]:
#Apply replace contractions function to cleaned_text column in df_tweets dataframe
df_tweets['cleaned_text'] = df_tweets['cleaned_text'].apply(lambda x: replace_contractions(x))

In [21]:
# function for twitter text cleaning 
def tweet_cleaner(text):
    # convert emojis to text
    text = emoji.demojize(text)
    text = text.replace(":"," ")
        # remove punctuations, numbers, and special characters 
        #text = re.sub("[^a-zA-Z]"," ",text) 
        # remove whitespaces 
    text = ' '.join(text.split()) 
        # convert text to lowercase
    text = text.lower()
    return text

In [22]:
df_tweets['cleaned_text'] = df_tweets['cleaned_text'].apply(lambda x: tweet_cleaner(x))

In [23]:
#Remove rows containing missing values under the cleaned_text column 
df_tweets = df_tweets[df_tweets['cleaned_text'].notnull()]

In [24]:
df_tweets.shape


(23000, 7)

In [30]:
df_tweets.to_csv (r'E:\twitter_SA\tweets\combined_tweets.csv', index = False, header=True)

In [28]:
#searching for tweets with 'giveaways'
new_df = df_tweets[~df_tweets['cleaned_text'].str.contains('giveaway', na=False)]
new_df.head()

,SN,Text,Username,Location,Date,Topic,cleaned_text
0,0,I am having a hard time with #Lent this year ...,wagagaikoko,NaN,2021-03-10 19:10:00,corona,i am having a hard time with #lent this year d...
1,1,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",RZgheib7,NaN,2021-03-10 19:10:00,corona,"#الطريق_مقطوعcitizens in lebanon, are facing d..."
2,2,I had the Corona already last January. So who...,cestlaviv,Carinthia - Austria,2021-03-10 19:10:00,corona,i had the corona already last january. so who ...
3,3,"Ooohh karma, you wonderful thing,man's restric...",FlavionsKimani,Nairobi Kenya,2021-03-10 19:10:00,corona,"ooohh karma, you wonderful thing,man's restric..."
4,4,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",Zizo90693787,NaN,2021-03-10 19:10:00,corona,"#الطريق_مقطوعcitizens in lebanon, are facing d..."


In [29]:
new_df.shape


(22964, 7)

In [32]:
new_df.duplicated(subset=['Text']).sum()

9302

In [33]:
final_df=new_df.drop_duplicates(subset=['Text'])

In [34]:
final_df.shape

(13662, 7)

In [35]:
final_df.to_csv (r'E:\twitter_SA\tweets\duplicate_removed.csv', index = False, header=True)

In [25]:
df_tweets.head()

,SN,Text,Username,Location,Date,Topic,cleaned_text
0,0,I am having a hard time with #Lent this year ...,wagagaikoko,NaN,2021-03-10 19:10:00,corona,i am having a hard time with #lent this year d...
1,1,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",RZgheib7,NaN,2021-03-10 19:10:00,corona,"#الطريق_مقطوعcitizens in lebanon, are facing d..."
2,2,I had the Corona already last January. So who...,cestlaviv,Carinthia - Austria,2021-03-10 19:10:00,corona,i had the corona already last january. so who ...
3,3,"Ooohh karma, you wonderful thing,man's restric...",FlavionsKimani,Nairobi Kenya,2021-03-10 19:10:00,corona,"ooohh karma, you wonderful thing,man's restric..."
4,4,"#الطريق_مقطوعCitizens in Lebanon, are facing ...",Zizo90693787,NaN,2021-03-10 19:10:00,corona,"#الطريق_مقطوعcitizens in lebanon, are facing d..."


In [26]:
df_tweets.tail()

,SN,Text,Username,Location,Date,Topic,cleaned_text
4995,4995,Some great random memories of life before loc...,MeadowsFestival,"The Meadows, Edinburgh",2021-03-09 16:34:00,lockdown,some great random memories of life before lock...
4996,4996,The First Female LockDown Housemate to own a ...,tosho_king,"Lagos, Nigeria",2021-03-09 16:34:00,lockdown,the first female lockdown housemate to own a h...
4997,4997,Happy Lockdown Period Anniversary everyone🥰🥰,_boujiee16_,Jamaica,2021-03-09 16:34:00,lockdown,happy lockdown period anniversary everyone smi...
4998,4998,Bindura – Gweshe Livelihoods DistributionsThe ...,care_agape,"Harare, Zimbabwe",2021-03-09 16:34:00,lockdown,bindura – gweshe livelihoods distributionsthe ...
4999,4999,2weeks till our ONE year lockdown anniversary ...,HouseKelly17,"Birkenhead, England",2021-03-09 16:34:00,lockdown,2weeks till our one year lockdown anniversary ...


In [35]:
df_tweets.to_csv (r'E:\twitter_SA\twitter_dataset_from_3mar-6mar\1stphase_tweets.csv', index = False, header=True)

In [ ]:
final_df.to_csv (r'pandemic_duplicate_removed.csv', index = False, header=True)